### Deep Convolutional GANs (DCGANs) for MNIST digits and fashion

#### Summary of how DCAGNs build on top of the basic GANs

1. Rather than fully-connected or pooling layers, we use strided convoltuions (for discriminator) and transposed convolutions (for generator). 
2. We use batch-normalization layers in both generator and discriminator to faster and more stable training. 
3. We use LeakyReLU as the alternative of ReLU to prevent the zero learning problem of ReLU. 

#### Git repo

In [ ]:
!git clone https://github.com/Daisuke0713/ANN-final.git
%cd ./content/ANN-final/VAEs

#### Libraries

In [2]:
import numpy as np
import time
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras, random, nn
from tensorflow import data
from keras import Model, Sequential
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, Conv2DTranspose, Reshape, Flatten, BatchNormalization, ReLU, LeakyReLU, Dropout, InputLayer

2023-04-14 11:49:31.750125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### GPU/CPU Settings

In [3]:
gpu_device = tf.test.gpu_device_name()
cpu_device = '/cpu:0'
# set CPU the device for now
device = gpu_device

2023-04-14 11:49:37.184945: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
if gpu_device != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(gpu_device))

#### Config Hyper-parameters

In [4]:
epochs = 10
batch_size = 32
latent_dim = 128
learning_rate = 1e-4

#### Data Pre-processing

In [5]:
# get data
(x_train_digits, _), (x_test_digits, _) = keras.datasets.mnist.load_data()

# process data
def preprocess_digits_image_data(data):
    # reshape, normalize, and binarize (gray) data
    shape = data.shape
    image_data = (data.reshape((shape[0], shape[1], shape[2], 1)) / 255. - 0.5) * 2.0
    return image_data.astype('float32')

x_train_digits = preprocess_digits_image_data(x_train_digits)
x_test_digits = preprocess_digits_image_data(x_test_digits)

#### Split into batches

In [6]:
def split_batch(image_data, batch_size):
    data_size = len(image_data)
    return (data.Dataset.from_tensor_slices(image_data).shuffle(data_size).batch(batch_size))

# split data into batches
x_train_digits = split_batch(image_data=x_train_digits, batch_size=batch_size)
x_test_digits  = split_batch(image_data=x_test_digits, batch_size=batch_size)

#### Define GANs

Generator

In [7]:
generator = Sequential([
    # input
    Dense(units=7*7*128, input_shape=(latent_dim,)),
    Reshape((7,7,128)),
    # conv 1
    Conv2DTranspose(filters=512, kernel_size=3, strides=2, padding='same'),
    BatchNormalization(),
    ReLU(max_value=0.2),
    # conv 2
    Conv2DTranspose(filters=512, kernel_size=3, strides=2, padding='same'),
    BatchNormalization(),
    ReLU(max_value=0.2),
    # final tanh    
    Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', activation='tanh')
])

Discriminator

In [8]:
discriminator = Sequential([
    # conv 1
    Conv2D(filters=512, kernel_size=3, strides=2, padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    LeakyReLU(0.2),
    # conv 2
    Conv2D(filters=512, kernel_size=3, strides=2, padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    LeakyReLU(0.2),
    # conv 3
    Conv2D(filters=512, kernel_size=3, strides=2, padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    LeakyReLU(0.2),
    # output
    Flatten(),
    Dense(units=1, activation='sigmoid')
])

Merge G and D

In [9]:
discriminator.compile(optimizer=Adam(learning_rate=learning_rate), loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
discriminator.trainable = False
gan = Sequential([
    generator, 
    discriminator
])
gan.compile(optimizer=Adam(learning_rate=learning_rate), loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])

#### Train GANs

In [ ]:
# labels
real = np.ones(shape=(batch_size, 1))
fake = np.zeros(shape=(batch_size, 1))

with tf.device(device_name=device):
    for epoch in range(epochs):
        for real_x in x_train_digits:
            # discriminator
            discriminator.trainable = True

            # train on real data
            d_loss_real = discriminator.train_on_batch(x=real_x, y=real)

            # train on fake data
            z = np.random.normal(loc=0, scale=1, size=(batch_size, latent_dim))
            fake_x = generator.predict_on_batch(x=z)
            d_loss_fake = discriminator.train_on_batch(x=fake_x, y=fake)
            d_loss = np.mean(d_loss_real + d_loss_fake)

            # generator
            discriminator.trainable = False
            g_loss = gan.train_on_batch(x=z, y=real)

        print(f'Epoch: {epoch}, Loss G: {g_loss[-1]}, Loss D: {d_loss}')

#### Generate images

In [ ]:
def generate():
    with tf.device(device):
        ramdom_vector_z = np.random.normal(loc=0, scale=1, size=(16, latent_dim))
        generated = generator(ramdom_vector_z)
        return generated

def save_img(imgs, filename):
    plt.figure(figsize=(4,4))
    for i in range(16):
        plt.subplot(4, 4, i + 1)
        plt.imshow(imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    # plt.savefig(f'./images/{filename}.png')

generated = generate()
save_img(generated, None)
